In [ ]:
#Import dependants
import pandas as pd
from pathlib import Path
import requests
import json
from pprint import pprint

from api_keys import geoapify_key

In [ ]:
# load and read the CSVs
clean_3_path = Path("Resources/clean_3.csv")
clean_3_df = pd.read_csv(clean_3_path)

clean_3_df.head()

In [3]:
# Function to geocode a city and state and update the DataFrame
def geocode_and_update(city, state, api_key):
    query = f"{city}, {state}"
    url = f"https://api.geoapify.com/v1/geocode/search?text={query}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
    return None

# Example usage: Loop through cities and states in the DataFrame and geocode
api_key = geoapify_key
clean_3_df['Latitude'] = None  # Create new columns for latitude and longitude
clean_3_df['Longitude'] = None
for index, row in clean_3_df.iterrows():
    city = row['City']
    state = row['State']
    coordinates = geocode_and_update(city, state, api_key)
    if coordinates:
        clean_3_df.at[index, 'Latitude'] = coordinates[1]  # Assuming Geoapify returns latitude first
        clean_3_df.at[index, 'Longitude'] = coordinates[0]
        print(f"Geocoded {city}, {state}")
    else:
        print(f"Failed to geocode {city}, {state}")

# View new DataFrame
clean_3_df.head()

In [ ]:
# Save and Output new DataFrame to new CSV file
clean_3_df.to_csv('Outputs/clean_3_updated.csv', index=False)